In [13]:
import sys, os, os.path
if not 'EXPERIMENT_HOME' in os.environ: # Configure local paths if not already defined
    ML_HOME = os.path.abspath(os.path.join("..", "ml_home")) # ML workspace
    EXPERIMENT_HOME = os.path.abspath(os.path.join(ML_HOME, "..")) # Experiment workspace
    DATA_HOME = os.path.abspath(os.path.join(EXPERIMENT_HOME,"dataset")) # Dataset location
    os.environ["EXPERIMENT_HOME"] = EXPERIMENT_HOME
    os.environ["DATA_HOME"] = DATA_HOME
    sys.path.insert(0, EXPERIMENT_HOME)
    sys.path.insert(0, ML_HOME) # Add to path so can load our library
    os.chdir(EXPERIMENT_HOME) # Change working directory to experiment workspace
print("Experiment Home: ", os.path.abspath(os.curdir), "; Data Home:", DATA_HOME)

Experiment Home:  /home/jovyan/dataset/Datastream ; Data Home: /home/jovyan/dataset


In [14]:
from datetime import datetime
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
import xlrd
import matplotlib.pyplot as plt
%matplotlib inline

/home/jovyan/dataset/Datastream


In [18]:
import sqlite3
DB_FILE = os.path.join(os.environ['DATA_HOME'], 'stock.db')
cnx = sqlite3.connect(DB_FILE)

In [4]:
os.chdir('/home/jovyan/dataset/Datastream')

In [7]:
import shutil
src = '/home/jovyan/dataset/Datastream/SET100_MissingData.xlsm'
dest = '/home/jovyan/dataset/Datastream/SET100_FullData.xlsm'
# shutil.copy2(src, dest)

'/home/jovyan/dataset/Datastream/SET100_FullData.xlsm'

In [8]:
!ls

DataDict_Static.xls  LBNGK100		     set100_daily_facts.csv.gz
DataTypes_DFO.xlsx   LBNGKSET		     SET100_FullData.xlsm
Data.xlsx	     set100_daily_facts.csv  SET100_MissingData.xlsm


In [24]:

fname = '/home/jovyan/dataset/Datastream/SET100_FullData.xlsm'

In [20]:
up = {}
for i in range(2,18):
    df_up = pd.read_sql("SELECT constituents FROM dataset_set100 WHERE update_id='Update_%s'" % i, cnx)
    stocks = set(df_up[0:1].values[0][0].replace(" ",",").split(','))
    up[i] = stocks

In [21]:
# What are the inclusion and exclusion for each update.
all_tickers = set()
idx = []
for k, v in up.items():
    id = 'Update_%s' % k
    try:
        exclusion = up[k-1].difference(up[k])
        inclusion = up[k].difference(up[k-1])
        all_tickers.update(v)
        print(id, exclusion, inclusion)
        idx.append({'update_seq': id,
                    'set100': ",".join(sorted(list(v))),
                    'inclusion': ",".join(sorted(list(inclusion))),
                    'exclusion': ",".join(sorted(list(exclusion)))
                   })
    except:
        pass

Update_3 {'THRE', 'BLA', 'MCOT', 'WHA', 'ESSO', 'SRICHA', 'RS', 'TASCO', 'NYT', 'DCC'} {'ICHI', 'PTG', 'SAWAD', 'DEMCO', 'SIM', 'SF', 'KTIS', 'SGP', 'HANA', 'ANAN'}
Update_4 {'HEMRAJ'} {'IFEC'}
Update_5 {'BIGC', 'PTG', 'BJC', 'BAY', 'NOK', 'STA', 'SIM', 'MEGA', 'IFEC', 'KTIS', 'THREL', 'SCCC', 'BCH'} {'BEAUTY', 'LHBANK', 'WHA', 'CBG', 'CKP', 'BA', 'UNIQ', 'SAPPE', 'RS', 'S', 'ASP', 'U', 'MONO'}
Update_6 {'GFPT', 'LOXLEY', 'GLOBAL', 'ASP', 'SAPPE', 'MC', 'U', 'TUF', 'DEMCO', 'SF', 'RATCH', 'SGP', 'PSL', 'MONO', 'ERW'} {'WORK', 'BLA', 'CHG', 'PTG', 'GPSC', 'SCN', 'PLAT', 'PLANB', 'TU', 'VNG', 'TASCO', 'EPG', 'SAMTEL', 'SCCC', 'GL'}
Update_7 {'BECL', 'BMCL', 'BGH'} {'BDMS', 'SGP', 'BEM'}
Update_8 {'JAS', 'M', 'UV', 'TICON', 'SCN', 'PLAT', 'SAMTEL', 'EARTH', 'SCCC'} {'GLOBAL', 'JWD', 'TRC', 'MTLS', 'TVO', 'IFEC', 'COM7', 'BCH', 'ERW'}
Update_9 {'PS'} {'PSH'}
Update_10 {'WORK', 'SVI', 'JWD', 'TRC', 'ERW', 'BJCHI', 'ANAN', 'GL'} {'SUPER', 'TKN', 'SCN', 'BIG', 'SPRC', 'VIBHA', 'THANI', 'KAMAR

In [23]:
len(all_tickers)

163

Create sheet  
https://stackoverflow.com/questions/19189048/writing-creating-a-worksheet-using-xlrd-and-xlwt-in-python



In [27]:
wb = xlrd.open_workbook(fname)
existing_sheets = wb.sheet_names()
existing_sheets

['REQUEST_TABLE',
 'BECL',
 'BIGC',
 'BMCL',
 'EARTH',
 'GLOW',
 'GOLD',
 'HEMRAJ',
 'IFEC',
 'ROBINS',
 'GL',
 'WHAUP',
 'JMART']

In [34]:
s = wb.sheets()[2] # Choose sheet by index to read data to data frame.

In [78]:
# Date index
s1 = pd.Series(s.col(0)[8:]) # observation starts at row 8
s1 = s1.apply(lambda x: datetime(*xlrd.xldate_as_tuple(x.value, wb.datemode)))
cols = {'Date':s1} # columns of data
for j in range(1, s.ncols):
    locCode = s.col(j)[3:4][0].value
    dataType = s.col(j)[4:5][0].value
    companyName = s.col(j)[5:6][0].value
    listedDate = datetime(*xlrd.xldate_as_tuple(s.col(j)[6:7][0].value, wb.datemode))
    c = s.col(j)
    # print("{}".format(c[3].value)) # Print each column in data file
    # Convert spreadsheet market cap value to python
    s2 = pd.Series(c[8:]) # observation starts at row 8
    s2 = s2.apply(lambda x: x.value)
    cols[dataType] = s2
df = pd.concat(cols, axis=1)

In [77]:
# s.col(0)[8:]  s.col(1)[8:]
df

,date,DY,EPS,MV,P,PH,PL,PO,VA,VO,PTBV,PE,PI,RI,MTBV,NOSH,DWTA,053E
0,2015-01-01,1.08,8.92,195524.9,237.0,NA,NA,NA,NA,NA,4.19,26.6,1351.7,2112.86,4.19,825000.0,102088552.0,0.00
1,2015-01-02,1.08,8.92,195524.9,237.0,NA,NA,NA,NA,NA,4.19,26.6,1351.7,2112.86,4.19,825000.0,102088552.0,0.00
2,2015-01-05,1.05,8.92,199649.9,242.0,242,233,233,86873.9,363.4,4.28,27.1,1380.2,2157.44,4.28,825000.0,102088552.0,0.40
3,2015-01-06,1.07,8.92,196349.9,238.0,242,236,240,90008.1,376.8,4.21,26.7,1357.4,2121.78,4.21,825000.0,102088552.0,0.39
4,2015-01-07,1.05,8.92,201299.9,244.0,245,237,237,41911.4,172.8,4.31,27.4,1391.6,2175.27,4.31,825000.0,102088552.0,0.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,2019-12-25,0.96,7.09,163349.9,198.0,NA,NA,NA,NA,NA,3.23,27.9,1129.2,1819.24,3.23,825000.0,91481022.0,0.00
1300,2019-12-26,0.96,7.09,163349.9,198.0,NA,NA,NA,NA,NA,3.23,27.9,1129.2,1819.24,3.23,825000.0,91481022.0,0.00
1301,2019-12-27,0.96,7.09,163349.9,198.0,NA,NA,NA,NA,NA,3.23,27.9,1129.2,1819.24,3.23,825000.0,91481022.0,0.00
1302,2019-12-30,0.96,7.09,163349.9,198.0,NA,NA,NA,NA,NA,3.23,27.9,1129.2,1819.24,3.23,825000.0,91481022.0,0.00


In [73]:
companyName

'BIG C SUPERCENTER DEAD - DELIST.28/09/17'

In [81]:
for t in sorted(all_tickers):
    b = "='%s'!$A$1" % t
    print(b)

='AAV'!$A$1
='ADVANC'!$A$1
='AEONTS'!$A$1
='AMATA'!$A$1
='ANAN'!$A$1
='AOT'!$A$1
='AP'!$A$1
='ASP'!$A$1
='AWC'!$A$1
='BA'!$A$1
='BANPU'!$A$1
='BAY'!$A$1
='BBL'!$A$1
='BCH'!$A$1
='BCP'!$A$1
='BCPG'!$A$1
='BDMS'!$A$1
='BEAUTY'!$A$1
='BEC'!$A$1
='BECL'!$A$1
='BEM'!$A$1
='BGH'!$A$1
='BGRIM'!$A$1
='BH'!$A$1
='BIG'!$A$1
='BIGC'!$A$1
='BJC'!$A$1
='BJCHI'!$A$1
='BLA'!$A$1
='BLAND'!$A$1
='BMCL'!$A$1
='BPP'!$A$1
='BTS'!$A$1
='CBG'!$A$1
='CENTEL'!$A$1
='CHG'!$A$1
='CK'!$A$1
='CKP'!$A$1
='COM7'!$A$1
='CPALL'!$A$1
='CPF'!$A$1
='CPN'!$A$1
='DELTA'!$A$1
='DEMCO'!$A$1
='DTAC'!$A$1
='EA'!$A$1
='EARTH'!$A$1
='EGCO'!$A$1
='EPG'!$A$1
='ERW'!$A$1
='ESSO'!$A$1
='GFPT'!$A$1
='GGC'!$A$1
='GL'!$A$1
='GLOBAL'!$A$1
='GLOW'!$A$1
='GOLD'!$A$1
='GPSC'!$A$1
='GULF'!$A$1
='GUNKUL'!$A$1
='HANA'!$A$1
='HEMRAJ'!$A$1
='HMPRO'!$A$1
='ICHI'!$A$1
='IFEC'!$A$1
='INTUCH'!$A$1
='IRPC'!$A$1
='ITD'!$A$1
='IVL'!$A$1
='JAS'!$A$1
='JMART'!$A$1
='JMT'!$A$1
='JWD'!$A$1
='KAMART'!$A$1
='KBANK'!$A$1
='KCE'!$A$1
='KKP'!$A$1
='KTB'!$A$1
